# Roboflow Guide : how to dowload back unannotated images

In [ ]:
import os
import time 
import roboflow
import requests
from datetime import datetime

rf = roboflow.Roboflow(api_key="Private_API_Key") # enter your Private API Key (go to the settings of your workspace, then to the API Keys)
project = rf.project("orange_morning_sunny") # enter your project id (click on the three dots of your project and do 'Copy Project Id')

records = []
for page in project.search_all(
    fields = ["id", "name", "owner"],
):
    records.extend(page)

print(f"{len(records)} images found")

360 images found


In [ ]:
SAVE_PATH = "/Users/baptistelucas/Downloads/final_roboflow/"    # your local path on computer
os.makedirs(SAVE_PATH, exist_ok=True)
downloaded_files = []
duplicate_handled = []

for record in records:
    base_url = "https://source.roboflow.com"
    url = f"{base_url}/{record['owner']}/{record['id']}/original.jpg"
    image_name = record['name']
    save_path = os.path.join(SAVE_PATH, image_name)

    if os.path.exists(save_path):           # if image already exists, handle duplicates by creating a folder
        folder_path = os.path.join(SAVE_PATH, os.path.splitext(image_name)[0])
        os.makedirs(folder_path, exist_ok=True)

        # move the existing image to the folder
        existing_image_path = os.path.join(folder_path, image_name)
        if not os.path.exists(existing_image_path):
            os.rename(save_path, existing_image_path)

        # save the new image that was creating a duplicate problem with a timestamp in the same folder
        timestamp = datetime.now().strftime('%Y%m%d%H%M%S')
        new_image_name = f"{os.path.splitext(image_name)[0]}_{timestamp}{os.path.splitext(image_name)[1]}"
        new_image_path = os.path.join(folder_path, new_image_name)

        try:
            response = requests.get(url)
            response.raise_for_status()

            with open(new_image_path, 'wb') as f:
                f.write(response.content)

            duplicate_handled.append(image_name)
            print(f"Handled duplicate: {image_name} -> Saved new version as {new_image_name} in {folder_path}")

        except requests.exceptions.RequestException as e:
            print(f"Error downloading duplicate image {image_name}: {e}")

    else:
        try:
            response = requests.get(url)
            response.raise_for_status()

            with open(save_path, 'wb') as f:
                f.write(response.content)

            downloaded_files.append(image_name)
            print(f"Downloaded: {image_name}")

        except requests.exceptions.RequestException as e:
            print(f"Error downloading image {image_name}: {e}")

print("\nSummary:")
print(f"Total Records: {len(records)}")
print(f"Successfully Downloaded: {len(downloaded_files)}")
print(f"Duplicates Handled: {len(duplicate_handled)}")


## original code

In [ ]:
SAVE_PATH = "/Users/baptistelucas/Downloads/finalbis/" # your local path on computer
for record in records:
        base_url = "https://source.roboflow.com"
        url = f"{base_url}/{record['owner']}/{record['id']}/original.jpg"

        try:
            response = requests.get(url)
            response.raise_for_status()

            os.path.join(SAVE_PATH, record['name'])
            with open(save_path, 'wb') as f:
                f.write(response.content)

            print(f"Downloaded: {record['name']}")

        except requests.exceptions.RequestException as e:
            print(f"Error downloading image: {e}")

# Reference

https://docs.roboflow.com/datasets/exporting-data#image-compression